In [ ]:
datadir = "../data/"
datafilename = "Certainty.xlsx"
datafilepath = datadir + datafilename

In [ ]:
from itertools import combinations
import pandas as pd
import numpy as np

# Read in network information from sheet P0
# For now we only use the folloiwng columns in each sheet
cols_used = ['Link ID','ANODE','BNODE','A X_COORD','A Y_COORD','B X_COORD','B Y_COORD','Link Length(miles)','# of lanes-A','Capacity-A (veh/h)',
            'auto volume(2010)-A','AADT(2010)-A','Speed(mph)-A','VMT-A']
df_p0= pd.read_excel(datafilepath, sheet_name='P0',usecols=cols_used).dropna(subset=['Link ID'])

# # For now we assume all occurrences of the same link in the same sheet share the same attribute values
# df_p0_unique = df_p0.drop_duplicates(subset=['Link ID']).dropna(subset=['Link ID', 'A X_COORD', 'A Y_COORD', 'B X_COORD', 'B Y_COORD'])


## Create a list of sheet names ##

# List of the first 6 integers
integers = [1, 2, 3, 4, 5, 6]

# Function to generate combinations
def generate_combinations(integers):
    all_combinations = []
    # Loop through lengths from 1 to 6
    for length in range(1, 7):
        # Generate combinations of the current length
        comb = combinations(integers, length)
        # Convert each combination to a string, add "p", and add to the list
        all_combinations.extend(['P' + ''.join(map(str, c)) for c in comb])
    return all_combinations

# Generate all combinations
sheet_names = generate_combinations(integers)

# Insert "P0" at the beginning of the list
sheet_names.insert(0, "P0")



In [ ]:

import torch
import torch.nn.functional as F
from torch_geometric.nn import EdgeConv
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

class EdgeLabelGNN(torch.nn.Module):
    def __init__(self):
        super(EdgeLabelGNN, self).__init__()
        self.conv1 = EdgeConv(nn=torch.nn.Sequential(
            torch.nn.Linear(2 * 16, 32),
            torch.nn.ReLU(),
            torch.nn.Linear(32, 16)
        ))
        self.fc = torch.nn.Linear(16, 1)

    def forward(self, x, edge_index, edge_attr):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.fc(x)
        return x





######### Nodes ##############
# colums that contain info about nodes in the graph
node_info_cols = ['ANODE','BNODE','A X_COORD','A Y_COORD','B X_COORD','B Y_COORD']
df_nodes= pd.read_excel(datafilepath, sheet_name='P0',usecols=node_info_cols).dropna(subset=['ANODE'])
# unique nodes among ANODE
df_nodes_ANODE_unique = df_nodes.drop_duplicates(subset=['ANODE'])
# unique nodes among BNODE
df_nodes_BNODE_unique = df_nodes.drop_duplicates(subset=['BNODE'])

# index of unique nodes in ANODE
ANODE_ind = df_nodes_ANODE_unique.index
# X-coord and Y-coord of nodes listed as ANODES 
Anodes = np.array(df_p0.iloc[ANODE_ind-1].loc[:,['ANODE','A X_COORD','A Y_COORD']]).astype(float)

# index of unique nodes in BNODE
BNODE_ind = df_nodes_BNODE_unique.index
# X-coord and Y-coord of nodes listed as BNODES 
Bnodes = np.array(df_p0.iloc[BNODE_ind-1].loc[:,['BNODE','B X_COORD','B Y_COORD']]).astype(float)

# combine Anodes and Bnodes
nodes = np.concatenate((Anodes, Bnodes))
# combine the two index arrays into one, without repetition
nodes_pd = pd.DataFrame(nodes)
nodes_unique = nodes_pd.drop_duplicates(subset=[0])
# Reset the index
nodes_unique.reset_index(drop=True, inplace=True)

# convert nodes_unique to np array
nodes_unique_arr = np.array(nodes_unique)
x =  torch.from_numpy(nodes_unique_arr).to(dtype=torch.float)


######### Edge Index #########
edge_pairs = np.column_stack((df_p0['ANODE'],df_p0['BNODE']))
edge_index = []
for ii in range(len(edge_pairs)):
    edge_node_ind = [nodes_unique[nodes_unique[0] == edge_pairs[ii,0]].index[0], nodes_unique[nodes_unique[0] == edge_pairs[ii,1]].index[0]]
    # edge_index[:,ii]= edge_node_ind
    edge_index.append(edge_node_ind)
edge_index = torch.tensor(edge_index).t().contiguous() 



# Prepare the dataset 
data = []
for ii in range(len(sheet_names)):
    # read the correct sheet 
    df= pd.read_excel(datafilepath, sheet_name=sheet_names[ii],usecols=cols_used).dropna(subset=['Link ID'])
    
    # dataframe for edge attributes
    df_edge_attr= np.column_stack((df['# of lanes-A'],df['Capacity-A (veh/h)'], df['Speed(mph)-A'],df['auto volume(2010)-A']))
    edge_attr = torch.tensor(df_edge_attr, dtype=torch.float)
    
    labels = torch.tensor(df['AADT(2010)-A'].to_numpy(),dtype=torch.float)
    new_data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=labels)
    data.append(new_data)




In [ ]:
# Define EdgeConv layer and network for predicting edge labels
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Define the EdgeConv layer with input and output dimensions
        self.conv1 = EdgeConv(nn=torch.nn.Sequential(
            torch.nn.Linear(2 * 3, 64),  # 3 features per node, concatenated for 6 features
            torch.nn.ReLU(),
            torch.nn.Linear(64, 32),
            torch.nn.ReLU()
        ))
        self.conv2 = EdgeConv(nn=torch.nn.Sequential(
            torch.nn.Linear(2 * 32, 64),  # 32 features per node from previous layer
            torch.nn.ReLU(),
            torch.nn.Linear(64, 32),
            torch.nn.ReLU()
        ))
        
        # Final layer for edge label prediction
        self.edge_predictor = torch.nn.Sequential(
            torch.nn.Linear(32 +32+ 4, 64),  # Concatenate node features with edge attributes
            torch.nn.ReLU(),
            torch.nn.Linear(64, 1)  # Output one label per edge
        )


    def forward(self, x, edge_index, edge_attr):
        # x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
        
        # Apply EdgeConv layers
        x = self.conv1(x, edge_index)
        x = self.conv2(x, edge_index)
        
        # Gather edge features by concatenating node features and edge attributes
        row, col = edge_index
        edge_features = torch.cat([x[row], x[col], edge_attr], dim=1)
        
        # Predict edge labels
        edge_labels = self.edge_predictor(edge_features).squeeze()
        
        return edge_labels

# Create DataLoader
loader = DataLoader([data], batch_size=1)

# Initialize model, optimizer, and loss function
model = Net()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()

# Training loop
for epoch in range(1000):
    for batch in loader:
        optimizer.zero_grad()
        out = model(x, edge_index, edge_attr)
        loss = criterion(out, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch}, Absolute Loss: {format(loss.item(),".2e")} ')




In [ ]:
labels_tensor = criterion(torch.zeros(torch.Tensor.size(labels)), labels)
loss_relative = loss/labels_tensor

print("\n Relative Loss = ", format(loss.item(), '.2e'), " / ", format(labels_tensor.item(),'.2e'), " = ", format(loss_relative.item(),'.2e'), "\n")


In [ ]:
print("\n Predicted AADT(after) = ", out)

In [ ]:
# # Test out preparing for data
# import torch
# from torch_geometric.data import Data


# # colums that contain info about nodes in the graph
# node_info_cols = ['ANODE','BNODE','A X_COORD','A Y_COORD','B X_COORD','B Y_COORD']
# df_nodes= pd.read_excel(datafilepath, sheet_name='P0',usecols=node_info_cols).dropna(subset=['ANODE'])
# # unique nodes among ANODE
# df_nodes_ANODE_unique = df_nodes.drop_duplicates(subset=['ANODE'])
# # unique nodes among BNODE
# df_nodes_BNODE_unique = df_nodes.drop_duplicates(subset=['BNODE'])
# # index of unique nodes in ANODE
# ANODE_ind = df_nodes_ANODE_unique.index
# # index of unique nodes in BNODE
# BNODE_ind = df_nodes_BNODE_unique.index
# # combine the two index arrays into one, without repetition
# nodes_ind=np.unique(np.concatenate((ANODE_ind, BNODE_ind)))


# ### select rows included in nodes_ind and all the columns to be included in nodes.
# nodes = np.array(df_p0.iloc[nodes_ind-1].loc[:,node_info_cols]).astype(float)
# x =  torch.from_numpy(nodes)

# ###########


# # edge index (ANODE)
# edge_index = torch.tensor([df_p0['ANODE'].to_numpy(),df_p0['BNODE'].to_numpy()], dtype=torch.long).t().contiguous()


# # Prepare the dataset 
# dataset = []
# for ii in range(len(sheet_names)):
#     # read the correct sheet 
#     df= pd.read_excel(datafilepath, sheet_name=sheet_names[ii],usecols=cols_used).dropna(subset=['Link ID'])

#     edge_attr = torch.tensor([df['# of lanes-A'].to_numpy(), df['Capacity-A (veh/h)'].to_numpy()], dtype=torch.float)
#     labels = torch.tensor([df['auto volume(2010)-A'].to_numpy()],dtype=torch.float)
#     data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=labels)
#     dataset.append(data)


In [ ]:
# import torch
# from torch_geometric.data import Data

# ######### Nodes ##############
# # colums that contain info about nodes in the graph
# node_info_cols = ['ANODE','BNODE','A X_COORD','A Y_COORD','B X_COORD','B Y_COORD']
# df_nodes= pd.read_excel(datafilepath, sheet_name='P0',usecols=node_info_cols).dropna(subset=['ANODE'])
# # unique nodes among ANODE
# df_nodes_ANODE_unique = df_nodes.drop_duplicates(subset=['ANODE'])
# # unique nodes among BNODE
# df_nodes_BNODE_unique = df_nodes.drop_duplicates(subset=['BNODE'])

# # index of unique nodes in ANODE
# ANODE_ind = df_nodes_ANODE_unique.index
# # X-coord and Y-coord of nodes listed as ANODES 
# Anodes = np.array(df_p0.iloc[ANODE_ind-1].loc[:,['ANODE','A X_COORD','A Y_COORD']]).astype(float)

# # index of unique nodes in BNODE
# BNODE_ind = df_nodes_BNODE_unique.index
# # X-coord and Y-coord of nodes listed as BNODES 
# Bnodes = np.array(df_p0.iloc[BNODE_ind-1].loc[:,['BNODE','B X_COORD','B Y_COORD']]).astype(float)

# # combine Anodes and Bnodes
# nodes = np.concatenate((Anodes, Bnodes))
# # combine the two index arrays into one, without repetition
# nodes_pd = pd.DataFrame(nodes)
# nodes_unique = nodes_pd.drop_duplicates(subset=[0])

# # convert nodes_unique to np array
# nodes_unique_arr = np.array(nodes_unique)
# x =  torch.from_numpy(nodes_unique_arr)


# ######### Edge Index #########
# edge_pairs = np.column_stack((df_p0['ANODE'],df_p0['BNODE']))
# edge_index = []
# for ii in range(len(edge_pairs)):
#     edge_node_ind = [nodes_unique[nodes_unique[0] == edge_pairs[ii,0]].index[0], nodes_unique[nodes_unique[0] == edge_pairs[ii,1]].index[0]]
#     # edge_index[:,ii]= edge_node_ind
#     edge_index.append(edge_node_ind)
# edge_index = torch.tensor(edge_index).t().contiguous() 

# ######## Edge Attributes ########

# ######## Edge Label #########

In [ ]:
# class EdgeLabelGNN(torch.nn.Module):
#     def __init__(self):
#         super(EdgeLabelGNN, self).__init__()
#         self.conv1 = EdgeConv(nn=torch.nn.Sequential(
#             torch.nn.Linear(2 * 3, 64),
#             torch.nn.ReLU(),
#             torch.nn.Linear(64, 32)
#         ))
#         self.fc = torch.nn.Linear(16, 1)

#     def forward(self, x, edge_index, edge_attr):
#         x = self.conv1(x, edge_index)
#         x = F.relu(x)
#         x = self.fc(x)
#         return x



# # Training loop
# for epoch in range(100):
#     for batch in loader:
#         optimizer.zero_grad()
#         # out = model(x,edge_index, edge_attr)
#         # loss = criterion(out, y)
#         out = model(batch.x, batch.edge_index, batch.edge_attr)
#         loss = criterion(out, batch.y)
#         loss.backward()
#         optimizer.step()
#     print(f'Epoch {epoch}, Loss: {loss.item()}')


In [ ]:
# Learning with EdgeConv, suggested by ChatGPT

# import torch
# import torch.nn.functional as F
# from torch_geometric.nn import EdgeConv
# from torch_geometric.data import Data, DataLoader

# class EdgeLabelGNN(torch.nn.Module):
#     def __init__(self):
#         super(EdgeLabelGNN, self).__init__()
#         self.conv1 = EdgeConv(nn=torch.nn.Sequential(
#             torch.nn.Linear(2 * 16, 32),
#             torch.nn.ReLU(),
#             torch.nn.Linear(32, 16)
#         ))
#         self.fc = torch.nn.Linear(16, 1)

#     def forward(self, x, edge_index, edge_attr):
#         x = self.conv1(x, edge_index)
#         x = F.relu(x)
#         x = self.fc(x)
#         return x

# # Example Data Preparation
# edge_index = torch.tensor([[0, 1], [1, 2]], dtype=torch.long).t().contiguous()
# edge_attr = torch.tensor([[3, 5], [2, 4]], dtype=torch.float)
# y = torch.tensor([1.0, 0.5], dtype=torch.float)

# data = Data(edge_index=edge_index, edge_attr=edge_attr, y=y)

# # Create DataLoader
# loader = DataLoader([data], batch_size=1)

# # Initialize model, optimizer, and loss function
# model = EdgeLabelGNN()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
# criterion = torch.nn.MSELoss()

# # Training loop
# for epoch in range(100):
#     for batch in loader:
#         optimizer.zero_grad()
#         out = model(batch.x, batch.edge_index, batch.edge_attr)
#         loss = criterion(out, batch.y)
#         loss.backward()
#         optimizer.step()
#     print(f'Epoch {epoch}, Loss: {loss.item()}')